# Movie-recommendation

In [1]:
from pyspark.sql import SparkSession
MAX_MEMORY = '10g'
spark = SparkSession.builder.appName('movie-recommendation')\
.config('spark.executor.memory', MAX_MEMORY)\
.config('spark.driver.memory', MAX_MEMORY)\
.getOrCreate()

23/06/11 21:49:33 WARN Utils: Your hostname, Keemyoui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.79 instead (on interface en0)
23/06/11 21:49:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/11 21:49:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/11 21:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark = SparkSession.builder.appName('movie-recommendation').getOrCreate()
ratings_file = '/Users/keemyohaan/Desktop/001.Python/004. Study/009.FastCampus/003.DE/01-spark/data/ml-25m/ratings.csv'
ratings_df = spark.read.csv(f'file:///{ratings_file}', inferSchema=True, header=True)
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [3]:
ratings_df = ratings_df.select(['userId', 'movieId', 'rating'])
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [4]:
ratings_df.select('rating').describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423475|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [5]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [6]:
# 추천에 쓸 것은 ALS, 메모리 에러가 난다. 이때 사용자가 직접 메모리를 지정해 줄 수 있다.
from pyspark.ml.recommendation import ALS
als = ALS(
    maxIter=5, #최고 반복 횟수
    regParam=0.1,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    coldStartStrategy = 'drop' # 학습하지 못한 데이터를 만났을 때 어떻게 할 것인가? 개발자가 설정해주는 것 nan or drop
         )

model = als.fit(train_df)

23/06/11 21:50:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/11 21:50:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/06/11 21:50:12 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [7]:
predictions = model.transform(test_df)
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    31|   6620|   1.5| 2.5205398|
|    85|   1088|   2.0| 3.7801747|
|   137|   1645|   3.0| 3.0494752|
|   243|   1580|   3.0| 2.6549966|
|   321|   3175|   3.0| 3.2452977|
|   368|   1580|   3.5| 3.5956376|
|   368|  54190|   4.0| 3.5182621|
|   375|   1580|   2.5|  3.447546|
|   458|   1580|   3.5|  3.164287|
|   472|    471|   5.0| 3.6718054|
|   472|   1088|   4.0| 3.4037948|
|   597|   1580|   4.0| 3.7289007|
|   597|   1591|   2.0| 2.5048122|
|   613|   1580|   3.0| 3.3647447|
|   737|   3175|   5.0|  4.090239|
|   858|   1580|   4.0| 2.9985473|
|   876|   1580|   3.5| 3.0288904|
|   876|   1645|   4.5| 3.2729747|
|   918|    471|   3.5| 3.9436395|
|  1088|   1580|   4.0| 3.7829845|
+------+-------+------+----------+
only showing top 20 rows



In [9]:
predictions.select('rating', 'prediction').describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4993976|           4993976|
|   mean|3.5339976603812273|  3.42265460687011|
| stddev|1.0605241529441467|0.6447545880655112|
|    min|               0.5|         -4.054752|
|    max|               5.0|          6.497198|
+-------+------------------+------------------+



In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print(rmse)

0.8099589504242205


In [15]:
model.recommendForAllUsers(3).show()

#여기서 숫자 3은 추천하려는 항목의 수를 나타냅니다. 즉, 이 함수는 각 사용자에게 상위 3개의 추천 항목을 제공합니다.
# 따라서, 이 함수의 출력은 각 사용자 ID와 그 사용자에 대한 상위 3개 추천 항목의 목록을 포함하는 DataFrame이 될 것입니다.

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    31|[{193257, 3.96241...|
|    34|[{180323, 5.71903...|
|    53|[{193257, 6.82056...|
|    65|[{194434, 6.57959...|
|    78|[{203882, 6.40449...|
|    85|[{193257, 5.96859...|
|   108|[{194434, 5.37117...|
|   133|[{183947, 5.39723...|
|   137|[{203882, 5.62979...|
|   148|[{194434, 5.81134...|
|   155|[{194434, 6.06701...|
|   193|[{202231, 5.70958...|
|   211|[{202231, 6.28785...|
|   243|[{159467, 4.64201...|
|   251|[{159467, 5.19484...|
|   255|[{194434, 6.01869...|
|   296|[{183947, 4.53659...|
|   321|[{194434, 5.59413...|
|   322|[{117352, 5.30781...|
|   362|[{194434, 5.67771...|
+------+--------------------+
only showing top 20 rows



In [16]:
model.recommendForAllItems(3).show() # 아이템마다 Top3 사람을 추천해줌

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     31|[{87426, 5.222674...|
|     34|[{32202, 5.282521...|
|     53|[{18885, 5.469643...|
|     65|[{87426, 5.256643...|
|     78|[{149507, 4.67973...|
|     85|[{105801, 4.84626...|
|    108|[{113621, 4.84933...|
|    133|[{86668, 4.937959...|
|    137|[{113441, 5.07438...|
|    148|[{142811, 3.97046...|
|    155|[{105801, 5.03413...|
|    193|[{87426, 5.028076...|
|    211|[{105801, 4.97279...|
|    243|[{143282, 4.85350...|
|    251|[{87426, 4.747472...|
|    255|[{87426, 4.749002...|
|    296|[{104135, 5.65779...|
|    321|[{105801, 5.09203...|
|    322|[{105801, 5.07379...|
|    362|[{18230, 5.062369...|
+-------+--------------------+
only showing top 20 rows



In [18]:
# 유저 한 3명에 대해서 만들어보자
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
user_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')

user_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [25]:
# 모델에 넣어주면 된다
## userId별로 추천 영화 5개 
user_recs = model.recommendForUserSubset(user_df, 5)

In [26]:
# 영화 아이디를 이름을 바꿔줘
movie_list = user_recs.collect()[0].recommendations

In [27]:
recs_df = spark.createDataFrame(movie_list) 
recs_df.show()

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 194434| 6.579591751098633|
| 193918| 6.579591751098633|
| 139036| 6.376902103424072|
| 169606| 6.283592224121094|
|  98221|6.1920552253723145|
+-------+------------------+



In [30]:
movies_file = '/Users/keemyohaan/Desktop/001.Python/004. Study/009.FastCampus/003.DE/01-spark/data/ml-25m/movies.csv'
movies_df = spark.read.csv(f'file:///{movies_file}', inferSchema=True, header=True)

In [31]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [32]:
recs_df.createOrReplaceTempView('recommendations')
movies_df.createOrReplaceTempView('movies')

In [33]:
query = """
SELECT * 
FROM 
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY 
    rating desc
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+------------------+-------+------------------+
|movieId|               title|            genres|movieId|            rating|
+-------+--------------------+------------------+-------+------------------+
| 193918|Smoke 'Em If You ...|            Comedy| 193918| 6.579591751098633|
| 194434|   Adrenaline (1990)|(no genres listed)| 194434| 6.579591751098633|
| 139036|World Gone Wild (...|     Action|Sci-Fi| 139036| 6.376902103424072|
| 169606|Dara O'Briain Cro...|            Comedy| 169606| 6.283592224121094|
|  98221|Year One, The (L'...|            Comedy|  98221|6.1920552253723145|
+-------+--------------------+------------------+-------+------------------+



실제로 쓸 때는, 하나의 Function안에 넣어서 쓰면 편리하게 쓸 수 있을 것 같다.

In [34]:
def get_recommendations(user_id, num_recs):
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF('userId')
    user_recs_df = model.recommendForUserSubset(user_df, num_recs) #recs_df랑 똑같은 형태가 되겠지?
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list) 
    
    recommended_movies = spark.sql(query)
    return recommended_movies
    

In [35]:
recs = get_recommendations(456, 10) 
recs.toPandas()

,movieId,title,genres,movieId,rating
0,194434,Adrenaline (1990),(no genres listed),194434,6.579592
1,193918,Smoke 'Em If You Got 'Em (1988),Comedy,193918,6.579592
2,139036,World Gone Wild (1988),Action|Sci-Fi,139036,6.376902
3,169606,Dara O'Briain Crowd Tickler (2015),Comedy,169606,6.283592
4,98221,"Year One, The (L'an 01) (1973)",Comedy,98221,6.192055


In [36]:
spark.stop()

In [ ]:
-